# MLOps Walkthrough - Model Evaluation

key principles / tasks supported by RSEs
* reusable components - making model results available to other researchers
* running at scale - serving many results
* interactive exploration of results (predictions and XAI)



## Example Load in trained model and do inference

use mlflow for iinference

In [62]:
import datetime
import os
import pathlib

In [1]:
import intake

In [79]:
import numpy
import pandas

In [74]:
import sklearn
import sklearn.preprocessing
import sklearn.metrics

### Get train and test data

In [63]:
try:
    rse_root_data_dir = pathlib.Path(os.environ['RSE22_ROOT_DATA_DIR'])
    print('reading from environment variable')
except KeyError as ke1:
    rse_root_data_dir = pathlib.Path(os.environ['HOME'])  / 'data' / 'ukrse2022'
    print('using default path')
rse_root_data_dir

using default path


PosixPath('/Users/stephen.haddad/data/ukrse2022')

In [64]:
rotors_catalog = intake.open_catalog(rse_root_data_dir / 'rotors_catalog.yml')
rotors_catalog 

rotors_catalog:
  args:
    path: /Users/stephen.haddad/data/ukrse2022/rotors_catalog.yml
  description: ''
  driver: intake.catalog.local.YAMLFileCatalog
  metadata: {}


In [65]:
rotors_df = rotors_catalog['rotors_preprocessed'].read()

In [69]:
# one small bit of cleaning: ensuring the correct datetime type for our time feature
rotors_df['time'] = pandas.to_datetime(rotors_df['time'])
temp_feature_names = [f'air_temp_{i1}' for i1 in range(1,23)]
humidity_feature_names = [f'sh_{i1}' for i1 in range(1,23)]
wind_direction_feature_names = [f'winddir_{i1}' for i1 in range(1,23)]
wind_speed_feature_names = [f'windspd_{i1}' for i1 in range(1,23)]
u_wind_feature_names = [f'u_wind_{i1}' for i1 in range(1,23)]
v_wind_feature_names = [f'v_wind_{i1}' for i1 in range(1,23)]
target_feature_name = 'rotors_present'


In [70]:
train_df = rotors_df[rotors_df['time'] < datetime.datetime(2020,1,1,0,0)]
val_df = rotors_df[rotors_df['time'] > datetime.datetime(2020,1,1,0,0)]

In [71]:
input_feature_names = temp_feature_names + humidity_feature_names + u_wind_feature_names + v_wind_feature_names

In [75]:
preproc_dict = {}
for if1 in input_feature_names:
    scaler1 = sklearn.preprocessing.StandardScaler()
    scaler1.fit(train_df[[if1]])
    preproc_dict[if1] = scaler1

In [76]:
target_encoder = sklearn.preprocessing.LabelEncoder()
target_encoder.fit(train_df[[target_feature_name]])


/Users/stephen.haddad/opt/anaconda3/envs/ukrse2022_mlops_model_evaluation/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LabelEncoder()

In [77]:
def preproc_input(data_subset, pp_dict):
    return numpy.concatenate([scaler1.transform(data_subset[[if1]]) for if1,scaler1 in pp_dict.items()],axis=1)

def preproc_target(data_subset, enc1):
     return enc1.transform(data_subset[[target_feature_name]])

In [80]:
X_train = preproc_input(train_df, preproc_dict)
y_train = numpy.concatenate(
    [preproc_target(train_df, target_encoder).reshape((-1,1)),
    1.0 - (preproc_target(train_df, target_encoder).reshape((-1,1))),],
    axis=1
)

/Users/stephen.haddad/opt/anaconda3/envs/ukrse2022_mlops_model_evaluation/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [81]:
X_val = preproc_input(val_df, preproc_dict)
y_val = numpy.concatenate(
    [preproc_target(val_df, target_encoder).reshape((-1,1)),
    1.0 - (preproc_target(val_df, target_encoder).reshape((-1,1))),],
    axis=1
)

/Users/stephen.haddad/opt/anaconda3/envs/ukrse2022_mlops_model_evaluation/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


### Load in trained model
We will now get a model from our most recent run, which will use to do inference and calculate metrics.

In [82]:
import mlflow
import mlflow.models

In [83]:
mlflow_server_address = '127.0.0.1'
mlflow_server_port = 5001
mlflow_server_uri = f'http://{mlflow_server_address}:{mlflow_server_port:d}'
mlflow_server_uri    

'http://127.0.0.1:5001'

In [84]:
mlflow.set_tracking_uri(mlflow_server_uri)

In [85]:
rse_rotors_experiment_name = 'rse_mlops_demo_rotors'

In [86]:
rse_rotors_experiment = mlflow.get_experiment_by_name(rse_rotors_experiment_name)
rse_rotors_experiment

<Experiment: artifact_location='/Users/stephen.haddad/data/ukrse2022/artifacts/1', experiment_id='1', lifecycle_stage='active', name='rse_mlops_demo_rotors', tags={}>

In [87]:
latest_run_info  = sorted(mlflow.list_run_infos(rse_rotors_experiment.experiment_id), key=lambda run1: run1.end_time)[-1]
latest_run_info.run_id, datetime.datetime.fromtimestamp(latest_run_info.end_time / 1000)

('c5becc89c46e4cfd82441cc37ab6afbf',
 datetime.datetime(2022, 8, 17, 19, 12, 33, 80000))

In [88]:
rse_rotors_model = mlflow.keras.load_model(latest_run_info.artifact_uri + '/model')
rse_rotors_model

In [89]:
rse_rotors_model.predict(X_val)

array([[8.2105380e-06, 9.9999177e-01],
       [1.0245468e-04, 9.9989748e-01],
       [3.7846921e-04, 9.9962151e-01],
       ...,
       [4.4974138e-10, 1.0000000e+00],
       [4.1534731e-06, 9.9999583e-01],
       [3.7166660e-03, 9.9628335e-01]], dtype=float32)

### Using the MLFlow model wrapping interface

You can use the ML Flow models module to be able to use models from different frameworks interchangeably for inference purposes.

In [94]:
logged_model = 'runs:/c5becc89c46e4cfd82441cc37ab6afbf/model'



In [95]:
loaded_model = mlflow.pyfunc.load_model(logged_model)
loaded_model

/Users/stephen.haddad/opt/anaconda3/envs/ukrse2022_mlops_model_evaluation/lib/python3.8/site-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.keras
  run_id: c5becc89c46e4cfd82441cc37ab6afbf

In [97]:
loaded_model.predict(X_val)

array([[8.2105380e-06, 9.9999177e-01],
       [1.0245468e-04, 9.9989748e-01],
       [3.7846921e-04, 9.9962151e-01],
       ...,
       [4.4974138e-10, 1.0000000e+00],
       [4.1534731e-06, 9.9999583e-01],
       [3.7166660e-03, 9.9628335e-01]], dtype=float32)

In [106]:
latest_run_saved_model_uri = f'runs:/{latest_run_info.run_id}/model'
latest_run_saved_model_uri

'runs:/c5becc89c46e4cfd82441cc37ab6afbf/model'

In [104]:
mlflow.keras.load_model().predict(X_val)

array([[8.2105380e-06, 9.9999177e-01],
       [1.0245468e-04, 9.9989748e-01],
       [3.7846921e-04, 9.9962151e-01],
       ...,
       [4.4974138e-10, 1.0000000e+00],
       [4.1534731e-06, 9.9999583e-01],
       [3.7166660e-03, 9.9628335e-01]], dtype=float32)

### Working with the ML Flow Client object and ML Flow UI

Ways of working
* browse models in UI
* search models for relevant characteristic (e.g. highest/lowest value of metric)

In [91]:
mlflow_client = mlflow.tracking.MlflowClient(tracking_uri=mlflow_server_uri)
mlflow_client

In [118]:
mlflow_client.list_registered_models()[0].latest_versions[0]

<ModelVersion: creation_timestamp=1660921051253, current_stage='None', description='', last_updated_timestamp=1660921051253, name='rse_rotors_20220819', run_id='c5becc89c46e4cfd82441cc37ab6afbf', run_link='', source='/Users/stephen.haddad/data/ukrse2022/artifacts/1/c5becc89c46e4cfd82441cc37ab6afbf/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [123]:
[(m1.name, m1.version) for m1 in mlflow_client.list_registered_models()[0].latest_versions]

[('rse_rotors_20220819', '1')]

In [120]:
rotors_reg_model_name = 'rse_rotors_20220819'
rotors_reg_model_version = '1'
reg_model_artifact_uri = f'models:/{rotors_reg_model_name}/{rotors_reg_model_version}'
reg_model_artifact_uri

'models:/rse_rotors_20220819/1'

In [133]:
mlflow_client.get_registered_model(rotors_reg_model_name)

<RegisteredModel: creation_timestamp=1660921051229, description='', last_updated_timestamp=1660921051253, latest_versions=[<ModelVersion: creation_timestamp=1660921051253, current_stage='None', description='', last_updated_timestamp=1660921051253, name='rse_rotors_20220819', run_id='c5becc89c46e4cfd82441cc37ab6afbf', run_link='', source='/Users/stephen.haddad/data/ukrse2022/artifacts/1/c5becc89c46e4cfd82441cc37ab6afbf/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>], name='rse_rotors_20220819', tags={}>

In [51]:
latest_run_info.artifact_uri

'/Users/stephen.haddad/data/ukrse2022/artifacts/1/c5becc89c46e4cfd82441cc37ab6afbf/artifacts'

In [128]:
mlflow.tracking.MlflowClient().get_run(latest_run_info.run_id)

<Run: data=<RunData: metrics={'loss': 0.017498191446065903,
 'root_mean_squared_error': 0.13228072226047516,
 'val_loss': 0.03716700151562691,
 'val_root_mean_squared_error': 0.19278743863105774}, params={'batch_size': '1000',
 'class_weight': 'None',
 'epochs': '100',
 'initial_epoch': '0',
 'max_queue_size': '10',
 'opt_amsgrad': 'False',
 'opt_beta_1': '0.9',
 'opt_beta_2': '0.999',
 'opt_decay': '0.0',
 'opt_epsilon': '1e-07',
 'opt_learning_rate': '0.0001',
 'opt_name': 'Adam',
 'sample_weight': 'None',
 'shuffle': 'True',
 'steps_per_epoch': 'None',
 'use_multiprocessing': 'False',
 'validation_batch_size': 'None',
 'validation_freq': '1',
 'validation_split': '0.0',
 'validation_steps': 'None',
 'workers': '1'}, tags={'mlflow.log-model.history': '[{"run_id": "c5becc89c46e4cfd82441cc37ab6afbf", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2022-08-17 18:12:24.662820", "flavors": '
                             '{"ker

# Example - calculate feature importance and vosualise results

### Example - Running a cluster with ray serve to serve many results

### Example - create a dashboard with holoviz that gets inference results from a ray serve or mlflow serve?

### Reference

* [Holoviz](https://holoviz.org/) 
* SHAP
* 